<a href="https://colab.research.google.com/github/Leo140102/desafioPython/blob/main/desafioPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install marvel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into accoun

In [4]:

from marvel import Marvel

PUBLIC_KEY = 'acfffd559153f10556f9159b27a35b57'
PRIVATE_KEY = '0def75c2d1f77394db0a53a96637e96a4ba0ebb3'

marvel = Marvel(PUBLIC_KEY=PUBLIC_KEY,
                PRIVATE_KEY=PRIVATE_KEY)

characters = marvel.characters
comics = marvel.comics
events = marvel.events



my_char = characters.all(nameStartsWith="Captain")["data"]["results"]

for char in my_char:
    print(char["id"], char["name"])
    for comic in char["comics"]["items"]:
        print(comic["name"])
    print("---------------------")

1009220 Captain America
A+X (2012) #1
A+X (2012) #4
A+X (2012) #9
A-Next (1998) #2
A-Next (1998) #3
A-Next (1998) #4
A-Next (1998) #5
A-Next (1998) #6
A-Next (1998) #7
A-Next (1998) #8
A-Next (1998) #9
A-Next (1998) #10
A-Next (1998) #11
A-Next (1998) #12
Adventures of Captain America (1991) #1
Adventures of Captain America (1991) #2
Adventures of Captain America (1991) #3
Adventures of Captain America (1991) #4
Age of Apocalypse (2012) #2 (Avengers Art Appreciation Variant)
Age of Heroes (2010) #1
---------------------
1010914 Captain America (House of M)
Captain America (2004) #10
Captain America: Winter Soldier Vol. 2 (Trade Paperback)
HOUSE OF M: WORLD OF M FEATURING WOLVERINE (Trade Paperback)
---------------------
1017295 Captain America (LEGO Marvel Super Heroes)
---------------------
1017327 Captain America (Marvel War of Heroes)
---------------------
1017575 Captain America (Sam Wilson)
All-New, All-Different Avengers (2015) #10
All-New, All-Different Avengers (2015) #11
All-N